In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
# import numpy as np
import pandas as pd

In [2]:
from create_matrix import *

from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
from degree_days import dds
appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "SanDiego"
year = 2014

import os
from degree_days import dds
import autograd.numpy as np




def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

In [3]:
def get_tensor(df):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    dfc = df.copy()

    df = dfc[energy_cols]

    tensor = df.values.reshape((len(df), 7, stop - start))
    return tensor


def create_region_df_dfc_static(region, year):
    df, dfc = create_matrix_single_region(region, year)
    tensor = get_tensor(df)
    static_region = df[['area', 'total_occupants', 'num_rooms']].copy()
    static_region['area'] = static_region['area'].div(4000)
    static_region['total_occupants'] = static_region['total_occupants'].div(8)
    static_region['num_rooms'] = static_region['num_rooms'].div(8)
    static_region =static_region.values
    return df, dfc, tensor, static_region



In [4]:
au_df, au_dfc, au_tensor, au_static = create_region_df_dfc_static('Austin', year)
sd_df, sd_dfc, sd_tensor, sd_static = create_region_df_dfc_static('SanDiego', year)

In [5]:
import pickle

def load_obj(name):
    with open(os.path.expanduser('~/git/' + name + '.pkl'), 'rb') as f:
        try:
            return pickle.load(f)
        except EOFError:
            return None

In [6]:
load_obj('pred_graph/AtoS/transfer/0-10.0-0')

{'Learning Params': {0: {'Iterations': 900,
   'Lambda': 0,
   'Least Train Error': 11.74557023366184,
   'Num home factors': 4,
   'Num season factors': 2}},
 'Predictions': {'aggregate': [              0           1           2           3           4            5   \
   54    227.370500  228.290730  222.337936  245.054879  351.716943   364.880028   
   203    90.375374   92.579328   78.322375  132.729495  388.185339   419.710952   
   527   599.301642  602.116373  583.908429  653.393246  979.642872  1019.905092   
   1450  434.440339  437.718391  416.513310  497.435747  877.388033   924.277658   
   
                  6            7            8           9           10  \
   54     396.615875   425.529592   424.310910  335.706981  272.214280   
   203    495.718365   564.966772   562.048027  349.841449  197.776312   
   527   1116.976233  1205.415262  1201.687652  930.672844  736.466309   
   1450  1037.327295  1140.323922  1135.982726  820.357243  594.183139   
   
               

In [61]:
from compiler.ast import flatten

    
for random_seed in range(5):
    for percentage in range(10, 100, 20):
        pred = {}
        for appliance in APPLIANCES_ORDER:
            pred[appliance] = []
        best_params_global = {}
        
        for parts in range(10):
            p = load_obj('pred_graph/AtoS/transfer/{}-{}-{}'.format(random_seed, float(percentage), parts))
            params = p['Learning Params']
            prediction = p['Predictions']
            best_params_global[parts] = params
            for appliance in APPLIANCES_ORDER:
                pred[appliance].append(prediction[appliance])
        
        for appliance in APPLIANCES_ORDER:
            pred[appliance] = pd.DataFrame(pd.concat(flatten(pred[appliance])))
        
        out = {'Predictions':pred, 'Learning Params':best_params_global}
        name = "{}-{}".format(random_seed, float(percentage))
        directory = os.path.expanduser('~/git/pred_graph/AtoS/transfer/')
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = os.path.expanduser('~/git/pred_graph/AtoS/transfer/'+ name + '.pkl')
        
        with open(filename, 'wb') as f:
            pickle.dump(out, f, pickle.HIGHEST_PROTOCOL)


In [62]:
from compiler.ast import flatten
out

{'Learning Params': {0: {0: {'Iterations': 500,
    'Lambda': 0.001,
    'Least Train Error': 11.262566933710309,
    'Num home factors': 3,
    'Num season factors': 2}},
  1: {1: {'Iterations': 900,
    'Lambda': 0,
    'Least Train Error': 11.521139984766325,
    'Num home factors': 5,
    'Num season factors': 4}},
  2: {2: {'Iterations': 1300,
    'Lambda': 0.01,
    'Least Train Error': 10.688433973361782,
    'Num home factors': 5,
    'Num season factors': 2}},
  3: {3: {'Iterations': 1300,
    'Lambda': 0.01,
    'Least Train Error': 10.905525590570004,
    'Num home factors': 5,
    'Num season factors': 2}},
  4: {4: {'Iterations': 500,
    'Lambda': 0.001,
    'Least Train Error': 9.93723805162396,
    'Num home factors': 4,
    'Num season factors': 2}},
  5: {5: {'Iterations': 1300,
    'Lambda': 0.01,
    'Least Train Error': 11.03259304076055,
    'Num home factors': 5,
    'Num season factors': 2}},
  6: {6: {'Iterations': 100,
    'Lambda': 0.001,
    'Least Train Err

In [28]:
for appliance in APPLIANCES_ORDER:
    pred[appliance] = pd.DataFrame(pd.concat(flatten(pred[appliance])))

In [57]:
p = load_obj('pred_graph/AtoS/transfer/{}-{}-{}'.format(random_seed, float(percentage), parts))
params = p['Learning Params']
prediction = p['Predictions']
best_params_global[parts] = params
for appliance in APPLIANCES_ORDER:
    pred[appliance].append(prediction[appliance])
pred

{'aggregate':               0           1           2           3           4           5   \
 54    233.408330  198.456830  226.180507  248.182523  339.369846  300.273897   
 203   269.601206  229.101353  261.226008  286.720720  392.383516  347.081305   
 527   584.480994  496.844111  566.358057  621.525593  850.167362  752.138746   
 1450  564.590941  479.803827  547.057324  600.430926  821.637740  726.796811   
 1524  165.878527  165.820506  162.761820  226.564127  684.805530  590.041761   
 1731  390.907875  334.506425  365.174774  387.427422  606.606477  511.468534   
 2031  115.419589  107.626223  110.682678  130.151894  269.414379  253.208268   
 2354  421.866691  363.188137  394.822272  414.555811  608.129364  538.307717   
 2606  331.123536  318.018734  323.273478  355.229544  530.703727  465.394343   
 3687  426.017249  413.847202  418.727129  448.403797  611.361559  550.710636   
 3864  137.670695  127.600255  131.638285  156.195059  291.038937  240.851665   
 3938  158.2620

In [33]:
k = load_obj('pred_graph/AtoS/normal/0-10.0')

In [37]:
k['Learning Params']

{0: {'Iterations': 100,
  'Least Train Error': 15.018973277785738,
  'Num home factors': 5,
  'Num season factors': 4,
  'lambda': 0.001},
 1: {'Iterations': 100,
  'Least Train Error': 7.645802748983202,
  'Num home factors': 4,
  'Num season factors': 3,
  'lambda': 0.001},
 2: {'Iterations': 1300,
  'Least Train Error': 13.256177471412311,
  'Num home factors': 4,
  'Num season factors': 3,
  'lambda': 0.001},
 3: {'Iterations': 100,
  'Least Train Error': 30.25032087413178,
  'Num home factors': 3,
  'Num season factors': 4,
  'lambda': 0.001},
 4: {'Iterations': 1300,
  'Least Train Error': 15.827594871661352,
  'Num home factors': 3,
  'Num season factors': 2,
  'lambda': 0.001},
 5: {'Iterations': 1300,
  'Least Train Error': 17.398701362330346,
  'Num home factors': 3,
  'Num season factors': 3,
  'lambda': 0.001},
 6: {'Iterations': 500,
  'Least Train Error': 13.723714072779178,
  'Num home factors': 4,
  'Num season factors': 2,
  'lambda': 0.001},
 7: {'Iterations': 1300,
 